In [ ]:
import pandas as pd
import calendar
from datetime import datetime
from tkinter import Tk, filedialog
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

# -------------------------
# Step 1. File selection
# -------------------------
Tk().withdraw()
file_path = filedialog.askopenfilename(
    title="Select ADWP Data File",
    filetypes=[("Excel files", "*.xlsx")]
)

if not file_path:
    raise SystemExit("❌ No file selected.")

print(f"Selected file: {file_path}")

# -------------------------
# Step 2. Dropdowns for Year & Month
# -------------------------
current_year = datetime.today().year

year_dd = widgets.Dropdown(
    options=[current_year - 1, current_year, current_year + 1],
    value=current_year,
    description="Year:"
)

month_dd = widgets.Dropdown(
    options=list(calendar.month_name)[1:],  # Jan..Dec
    value=calendar.month_name[datetime.today().month],
    description="Month:"
)

ok_button = widgets.Button(description="OK", button_style="success")
output_area = widgets.Output()

display(widgets.VBox([year_dd, month_dd, ok_button, output_area]))

# -------------------------
# Step 3. Processing function
# -------------------------
def process_adwp(b):
    with output_area:
        clear_output()
        
        # Read Excel
        df = pd.read_excel(file_path, sheet_name="Data", engine="openpyxl")
        df["Period Month"] = pd.to_datetime(df["Period Month"]).dt.to_period("M").dt.to_timestamp()
        
        # User selections
        sel_year = year_dd.value
        sel_month = list(calendar.month_name).index(month_dd.value)
        curr_date = datetime(sel_year, sel_month, 1)
        
        # 24-month horizon
        start_date = datetime(sel_year, 1, 1)
        horizon = pd.date_range(start=start_date, periods=24, freq="MS")
        month_labels = [d.strftime("%b-%y") for d in horizon]
        
        output_records = []

        # Define position key (PID or MPP)
        def get_key(row):
            if pd.notna(row["PID"]):
                return ("PID", row["PID"], row["GCB"], row["Business Framework"])
            else:
                return ("MPP", row["MPP ID"], row["GCB"], row["Business Framework"])

        grouped = df.groupby(df.apply(get_key, axis=1))

        for key, group in grouped:
            kind = key[0]
            group = group.sort_values("Period Month")
            
            # Prepare base record
            latest = group.iloc[-1]
            record = {
                "Business Service L2": latest.get("Business Service L2"),
                "Business Framework Group": latest.get("Business Framework Group"),
                "Business Framework": latest.get("Business Framework"),
                "Country R1": latest.get("Country R1"),
                "Country R2": latest.get("Country R2"),
                "Country R3": latest.get("Country R3"),
                "PID": latest.get("PID") if kind == "PID" else None,
                "GCB": latest.get("GCB"),
                "MPP ID": latest.get("MPP ID") if kind == "MPP" else None,
                "Stack": latest.get("Stack") if kind == "MPP" else None
            }

            # Initialize all months to None
            for label in month_labels:
                record[label] = None

            # Carry-forward logic
            last_fte = None
            for month in month_labels:
                month_dt = datetime.strptime(month, "%b-%y")
                # Check if group has FTE for this month
                row = group[group["Period Month"] == month_dt]
                if not row.empty:
                    last_fte = row["FTE"].iloc[0]
                # Fill FTE forward
                if last_fte is not None:
                    record[month] = last_fte
                # Mark FTE column as latest available <= current month
                if month_dt <= curr_date and record[month] is not None:
                    record["FTE"] = record[month]

            output_records.append(record)

        # Build DataFrame
        out_df = pd.DataFrame(output_records)

        # Sorting
        sort_cols = [
            "Business Service L2",
            "Business Framework Group",
            "Business Framework",
            "Country R1",
            "Country R2",
            "Country R3",
            "PID",
            "MPP ID",
            "GCB"
        ]
        out_df = out_df.sort_values(sort_cols, na_position="last").reset_index(drop=True)

        # Save output
        out_file = os.path.join(
            os.path.dirname(file_path),
            f"ADWP_Output_{sel_year}_{sel_month:02}.xlsx"
        )
        with pd.ExcelWriter(out_file, engine="openpyxl") as writer:
            out_df.to_excel(writer, sheet_name="Output", index=False)

        print(f"✅ ADWP Output written to: {out_file}")
        print(f"Rows: {len(out_df):,}, Columns: {len(out_df.columns)}")

ok_button.on_click(process_adwp)
